In [43]:
pip install keras-tuner

In [44]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [45]:
fashion_mnist=keras.datasets.fashion_mnist

In [46]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [47]:
train_images = train_images/255.0
test_images = test_images/255.0

In [48]:
train_images[0].shape

(28, 28)

In [49]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [53]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
      filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16), 
      kernel_size=hp.Choice('conv_1_kernel', values = [3,5]), 
      activation='relu',
      input_shape=(28,28,1)
      ), 
      keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
          kernel_size=hp.Choice('conv_2_kernel', values = [3,5]), 
          activation='relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
          activation='relu'
      ),
      keras.layers.Dense(10, activation='softmax')
    ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
 
  return model

In [54]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [56]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5, directory='output', project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json


In [57]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 12m 34s]
val_accuracy: 0.906000018119812

Best val_accuracy So Far: 0.9129999876022339
Total elapsed time: 01h 06m 58s
INFO:tensorflow:Oracle triggered exit


In [58]:
model = tuner_search.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        27696     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2654304   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 2,683,610
Trainable params: 2,683,610
Non-trainable params: 0
_________________________________________________________________
